# Gemini proxing demo
This is a demo of using CAPSEM to proxy requests to Gemini via the CAPSEM Proxy. It shows how to set up a simple agent that uses Gemini as its LLM, and how to enforce policies on the agent's behavior.

In [2]:
import time
from google import genai
from google.genai import types


## Instantiate a proxied Gemini client

To proxy the requests to Gemini throught CAPSEM, we need to  pass the the CAPSEM Proxy URL as part of the HTTP options when creating the Gemini client.

We start by defining the http options with the CAPSEM Proxy URL as `base_url` and then instantiate the Gemini client with these options.

In [3]:
CAPSEM_PROXY = "http://127.0.0.1:8000"
http_options = types.HttpOptions(base_url=CAPSEM_PROXY)
client = genai.Client(http_options=http_options)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


## Calling Gemini via CAPSEM Proxy 

We can now use the Gemini client as usual, and all requests will be proxied through CAPSEM. This allows us to enforce policies on the requests and responses, such as filtering out sensitive information or input sanitization. Here is a simple example of generating content with Gemini via CAPSEM Proxy.



In [11]:
def weather_function(location: str) -> dict:
    # Dummy implementation for illustration
    return {"temperature": "20°C", "location": location}

config = types.GenerateContentConfig(tools=[weather_function])

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="what is the weather in paris?",
    config=config,
)
print(response.text)

The weather in Paris is 20°C.


## example with PII detection in tool result

In this example, the agent calls a tool that returns PII information. CAPSEM proxy detects the PII in the tool result and blocks the response.

In [10]:
def weather_pii(location: str) -> dict:
    # Dummy implementation for illustration
    return {"temperature": "20°C", "location": "PII test@gmail.com"}

config = types.GenerateContentConfig(tools=[weather_pii])

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="what is the weather in paris?",
    config=config,
)
print(response.text)

ClientError: 403 None. {'detail': "Request blocked by security policy: PII detected in tool 'weather_pii' response: EMAIL_ADDRESS(count=1, score=1.00, action=BLOCK)"}